In [ ]:
from generate import *
from sage.modules.free_module_integer import IntegerLattice
from math import isclose
from random import seed
from LLL_implementation import full_LLL, full_LLL_printing

import time 

In [ ]:
def gaussian_reduction(b_1, b_2):
    """Input: vectors as a list
    Performs the Lagrange-Gauss algorithm.
    Output: sage vectors [longer, shorter]
    """
    b_1 = vector(b_1)
    b_2 = vector(b_2)
    if b_1.norm() < b_2.norm():
        b_1 , b_2 = b_2, b_1
    r = round(b_1.dot_product(b_2) / b_2.norm()**2)
    a = b_1 - r * b_2
    if a.norm() < b_2.norm():
        a, b_2 = gaussian_reduction(a, b_2)
    return a, b_2

def vector_to_int_list(vector) -> list:
    return [int(num) for num in vector.list()]

In [ ]:
def first_nonzero_norm_vector(vectors):
    for v in vectors:
        if euclidean_norm(v) != 0:
            return v

In [ ]:
def semaev(basis):
    dim = basis.nrows()

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    iterace = 0

    while True:
        shortest_vectors_sequence.append(basis[0].norm().n())
        iterace += 1
        
        # reduce the pair b_1 and b_2
        basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

        # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis)
        lc_cube = find_real_minimum_local(G, dim-1, 1) 
        slc = shortest_lc_in_cube(lc_cube, basis)
        candidate = vector(ZZ,slc) * basis

        # if I found a shorter vector candidate, continue
        longest_norm = sqrt(G[dim-1,dim-1]).n()
        if candidate.norm().n() >= longest_norm.n():
            break
        basis[dim-1] = candidate
        sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
        basis = basis[sorted_indices, :]

    return basis, iterace

def find_real_minimum_local(G, current_row, inserted_component) -> vector:
    dimension = len(matrix_to_list(G))
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([inserted_component * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right(-(inserted_component) * matrixB)).list()
    result.insert(current_row, inserted_component)
    return vector(result).n()




In [ ]:
# COUNTING SR +  RUNTIME 


def counter(dimension : int, perimeter, tries, function, jsonfilename):
    """
    Runs several iterations of an algorithm on a random lattice and returns its SR.
    """
    print("dimension", dimension)
    res = {"True":0, "False":0}
    total_time =0
    total_iterations = 0
    better_than_LLL_count = 0
    diff_from_exact=0
    matrices = []
    for _ in range(tries):
        the_matrix = random_special_matrix(dimension, perimeter)
        matrices.append(matrix_to_list(the_matrix))
        swapresult, this_it = function(the_matrix)
        total_iterations += float(this_it)
        this_function_closest_norm = euclidean_norm(matrix_to_list(swapresult)[0])
        L = IntegerLattice(the_matrix, lll_reduce=False)
        sv_exact = L.shortest_vector()
        sv_exact_norm = sv_exact.norm().n()
        lllnorm = euclidean_norm(first_nonzero_norm_vector(matrix_to_list(the_matrix.LLL())))

        if this_function_closest_norm < lllnorm:
            better_than_LLL_count +=1

        if  isclose(this_function_closest_norm, sv_exact_norm, abs_tol=1e-4):
            res["True"]+=1
            continue
        diff_from_exact += this_function_closest_norm/sv_exact_norm
        res["False"]+=1
    
    print("Average num of iterations", total_iterations/tries,"\t better than LLL",better_than_LLL_count, "\t\tof method ", str(function))
    if res["False"] > 0:
        print("\t\t vs from exact, if failed",diff_from_exact/res["False"])
    with open(jsonfilename, 'w') as f:
        json.dump(matrices, f)
    return res

In [ ]:
def semaev_modified_for_higher_bases(basis):
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    iterations = 0

    while True:
        iterations += 1

        # Reduce the pair b_1 and b_2
        basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

        # find a minimum of |a|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis)
        lc_cube = vector_to_list(find_real_minimum_local(G, dim - 1, 1))
        slc = shortest_lc_in_cube(lc_cube, basis)
        candidate = vector(ZZ,slc) * basis
        candidate_norm = candidate.norm().n() 

        # if I found a shorter vector candidate, continue
        longest_norm = sqrt(G[dim-1,dim-1]).n()

        # if I found a shorter vector a, continue
        if candidate_norm < longest_norm:

            basis[dim-1] = candidate
            sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
            basis = basis[sorted_indices, :]
            continue
        else: # cannot be shortened
            tmp_base, tmp_it = semaev(basis[0:dim-1])
            iterace += tmp_it

            # semaev 3x3 didnt change anything
            if basis[0:dim-1] == tmp_base:
                break
            basis[0:dim-1] = tmp_base
            sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
            basis = basis[sorted_indices, :]

    return basis, iterations






In [ ]:
def semaev_with_rounding(basis):
    dim = basis.nrows()

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    iterations = 0
    shortest_vectors_sequence = []
    
    while True:
        shortest_vectors_sequence.append(basis[0].norm().n())
        iterations += 1
        # reduce the pair b_1 and b_2

        basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

        # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis)

        lc_cube = find_real_minimum_local(G, dim-1, 1) 
 


        slc = vector([round(x) for x in lc_cube])
        candidate = vector(ZZ,slc) * basis

        # if I found a shorter vector candidate, continue
        longest_norm = sqrt(G[dim-1,dim-1]).n()
        if candidate.norm().n() >= longest_norm.n():
            break
        basis[dim-1] = candidate
        sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
        basis = basis[sorted_indices, :]

    return basis, iterations



In [ ]:
def single_semaev_round_then_LLL(basis):
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    # -- - - - - - SEMAEV

    # reduce the pair b_1 and b_2
    basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

    # find a minimum
    G = gram_matrix(basis)
    lc_cube = find_real_minimum_local(G, dim-1, 1) 
    slc = vector([round(x) for x in lc_cube])
    candidate = vector(ZZ,slc) * basis

    # substitute right away
    basis[dim-1] = candidate

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]

        
    basis, it = auxiliary_LLL(basis)

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    return basis, it


In [ ]:
def built_in_LLL(basis):
    dim = basis.nrows()
    basis = basis.LLL()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :] 
    return basis, 0
    

In [ ]:

def auxiliary_LLL(basis):
    basis, it = full_LLL_printing(basis)
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :] 
    return matrix(basis), it


In [ ]:
# table 5.2, iterations of semaev depending or range

for exp in range(5, 41, 5):
    print("exp",exp)
    print(counter(3, 10^exp, 100, semaev,"semaev_tests/table5-2_exponent_"+str(exp)+".json"))

### Results: table 5.2, iterations of semaev depending or range

exp 5
Average num of iterations 4.62 				of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

exp 10
Average num of iterations 6.95 				of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

exp 15
Average num of iterations 9.2 			of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

exp 20
Average num of iterations 11.16 	 		of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

exp 25
Average num of iterations 13.72 		of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

exp 30
Average num of iterations 15.66 		of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

exp 35
Average num of iterations 18.96 			of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

exp 40
Average num of iterations 21.28 			of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}

In [ ]:
# table 5.3, iterations of semaev with rounding depending or range

for exp in range(30, 41, 5):
    print("exp",exp)
    print(counter(3, 10^exp, 100, semaev_with_rounding,"semaev_tests/table5-3_rounding_exponent_"+str(exp)+".json"))

# table 5.3, iterations of semaev with rounding 3x3 depending or range

exp 5
dimension 3
Average num of iterations 4.55 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

exp 10
dimension 3
Average num of iterations 6.75 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

exp 15
dimension 3
Average num of iterations 9.33 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

exp 20
dimension 3
Average num of iterations 11.22 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

exp 25
dimension 3
Average num of iterations 13.39 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

exp 30
dimension 3
Average num of iterations 16.03 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

exp 35
dimension 3
Average num of iterations 19.29 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

exp 40
dimension 3
Average num of iterations 21.92 		 total diff from exact, if failed 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}


In [ ]:
# table 5.4, Semaev in higher

for dim in [4,6,10]:
    print(counter(dim, 10^10, 100, semaev,"semaev_tests/table5-4-semaev"+str(dim)+".json"))

### table 5.4, Semaev in higher

dimension 4
Average num of iterations 13.61 	 better than LLL 2 		of method  <function semaev at 0xffff673655a0>
{'True': 100, 'False': 0}


dimension 6
Average num of iterations 27.23 	 better than LLL 2 		of method  <function semaev at 0xffff673655a0>
		 vs from exact, if failed 1.03800879006567
{'True': 98, 'False': 2}
dimension 10


In [ ]:
# table 5.6 semaev modified for higher dimensions
print(counter(4, 10^10, 100, semaev_modified_for_higher_bases,"semaev_tests/table5-6-semaev-higher-dim4.json"))
print(counter(5, 10^10, 100, semaev_modified_for_higher_bases,"semaev_tests/table5-6-semaev-higher-dim5.json"))

### table 5.6 semaev modified for higher dimensions

dimension 4
Average num of iterations 15.83 	 better than LLL 2 		of method  <function semaev_modified_for_higher_bases at 0xffff11a27640>
{'True': 100, 'False': 0}


dimension 5
Average num of iterations 23.58 	 better than LLL 2 		of method  <function semaev_modified_for_higher_bases at 0xffff11a27640>

		 vs from exact, if failed 1.12608483914183
         
{'True': 99, 'False': 1}


In [ ]:
# table 5.7 semaev with rounding, higher dimensions


for dim in [3, 4, 6, 10]:
    print(counter(dim, 10^10, 100, semaev_with_rounding,"semaev_tests/table5-7-semaev-rounding-dim-"+str(dim)+".json"))


In [ ]:
print(counter(15, 10^10, 100, semaev_with_rounding,"semaev_tests/table5-7-semaev-rounding-dim-"+str(15)+".json"))


### table 5.7 semaev with rounding, higher dimensions

dimension 3
Average num of iterations 6.66 	 better than LLL 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
{'True': 100, 'False': 0}

dimension 4
Average num of iterations 15.34 	 better than LLL 3 		of method  <function semaev_with_rounding at 0xffff11a27370>
		 vs from exact, if failed 1.70571241590494
{'True': 98, 'False': 2}

dimension 6
Average num of iterations 39.34 	 better than LLL 1 		of method  <function semaev_with_rounding at 0xffff11a27370>
		 vs from exact, if failed 268.253798334372
{'True': 62, 'False': 38}

dimension 10
Average num of iterations 16.85 	 better than LLL 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
		 vs from exact, if failed 2410.84748963606
{'True': 0, 'False': 100}

dimension 15
Average num of iterations 9.63 	 better than LLL 0 		of method  <function semaev_with_rounding at 0xffff11a27370>
		 vs from exact, if failed 4386.87277005781
{'True': 0, 'False': 100}

In [ ]:
# table 6.1 LLL iterations

for dim in [3, 4, 6, 10, 15, 20]:
    print(counter(dim, 10^10, 100, auxiliary_LLL,"semaev_tests/table6-1-LLL-dim-"+str(dim)+".json"))


### table 6.1 LLL iterations

Average num of iterations 25.15 	 better than ex 0 		of method  <function auxiliary_LLL at 0xffff2e9729e0>
{'True': 100, 'False': 0}

dimension 4
Average num of iterations 47.62 	 better than LLL 0 		of method  <function auxiliary_LLL at 0xffff2e9729e0>
{'True': 100, 'False': 0}

dimension 6
Average num of iterations 109.9 	 better than LLL 0 		of method  <function auxiliary_LLL at 0xffff2e9729e0>
		 vs from exact, if failed 1.10936216782046
{'True': 99, 'False': 1}

dimension 10
Average num of iterations 279.55 	 better than LLL 0 		of method  <function auxiliary_LLL at 0xffff2e9729e0>
		 vs from exact, if failed 1.06240899435534
{'True': 95, 'False': 5}

dimension 15
Average num of iterations 514.22 	 better than LLL 4 		of method  <function auxiliary_LLL at 0xffff2e9729e0>
		 vs from exact, if failed 1.06240899435534
{'True': 80, 'False': 20}

dimension 15
Average num of iterations 718.64 	 better than LLL 4 		of method  <function auxiliary_LLL at 0xffff2e9729e0>
		 vs from exact, if failed 1.06240899435534
{'True': 57, 'False': 43}

In [ ]:
# table 6.2 proposed algorithm iterations accuracy

for dim in [3, 4, 6, 10, 15, 20]:
    print(counter(dim, 10^10, 100, single_semaev_round_then_LLL,"semaev_tests/table6-2-proposed-dim-"+str(dim)+".json"))


### table 6.2 proposed algorithm iterations accuracy

dimension 3
Average num of iterations 14.17 	 better than LLL 1 		of method  <function single_semaev_round_then_LLL at 0xffff2e972680>

{'True': 100, 'False': 0}

dimension 4
Average num of iterations 34.71 	 better than LLL 1 		of method  <function single_semaev_round_then_LLL at 0xffff2e972680>
		 vs from exact, if failed 1.03344167202749

{'True': 99, 'False': 1}

dimension 6
Average num of iterations 88.41 	 better than LLL 1 		of method  <function single_semaev_round_then_LLL at 0xffff2e972680>

{'True': 100, 'False': 0}

dimension 10
Average num of iterations 235.83 	 better than LLL 8 		of method  <function single_semaev_round_then_LLL at 0xffff2e972680>
		 vs from exact, if failed 1.09300400546257

{'True': 97, 'False': 3}

dimension 15
Average num of iterations 432.56 	 better than LLL 11 		of method  <function single_semaev_round_then_LLL at 0xffff2e972680>
		 vs from exact, if failed 1.08338526759259

{'True': 80, 'False': 20}
Average num of iterations 585.16 	 better than LLL 28 		of method  <function single_semaev_round_then_LLL at 0xffff2e972680>
		 vs from exact, if failed 1.09438345655423
{'True': 57, 'False': 43}